Create a ResNet18 for our simulation of a malicious network attack

code referenced from: \
https://github.com/samcw/ResNet18-Pytorch/blob/master/ResNet18.ipynb \
https://github.com/kuangliu/pytorch-cifar/blob/master/main.py \
Assignment 2 in class

In [1]:
import torch
import torchvision

from torchsummary import summary

import numpy as np

a bit to make sure we are using the right python environment \
and if cuda is available

In [2]:
import sys
sys.version

'3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:49:16) [MSC v.1929 64 bit (AMD64)]'

In [3]:
torch.cuda.is_available()

True

In [4]:
#check gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Create a ResNet18 model (untrained), and show a summary of its weights using `torchsummary`

In [5]:
res18 = torchvision.models.resnet18().cuda() if torch.cuda.is_available() else torchvision.models.resnet18()

In [6]:
# check if the parameters are in cuda
next(res18.parameters()).is_cuda

True

In [7]:
summary(res18, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

Train it using CIFAR10 dataset, initially without a backdoor

In [8]:
import torchvision.transforms as transforms

# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
# )

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [9]:
cifar10_train = torchvision.datasets.CIFAR10('datasets/cifar_10', download=True, transform=transform_train)
cifar10_test = torchvision.datasets.CIFAR10('datasets/cifar_10', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=200, shuffle=False, num_workers=4)

In [11]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [12]:
# copied from assignment 2
def compute_accuracy(prediction,gt_logits):
    pred_idx = np.argmax(prediction,1,keepdims=True)
    matches = pred_idx == gt_logits[:,None]
    acc = matches.mean()
    return acc

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(res18.parameters(),lr=0.005)
optimizer = torch.optim.SGD(res18.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [15]:
for epoch in range(100):
    res18.train()
    it = 0
    total_loss = 0
    total_acc = 0
    total_item_ct = 0
    # a very standard training loop
    for inputs, label in train_loader:
        # move data to cuda device
        inputs, label = inputs.to(device), label.to(device)
        
        optimizer.zero_grad()
        pred = res18(inputs)
        loss = loss_fn(pred,label)
        accuracy = compute_accuracy(pred.cpu().detach().numpy(),label.cpu().detach().numpy())
        # accuracy = compute_accuracy(pred,label)
        loss.backward()
        optimizer.step()

        # print(f'Epoch: {epoch}, Iteration: {it} | Loss: {loss.item()} | Accuracy: {accuracy}')
        it += 1
        total_loss += loss.item()
        total_acc += accuracy * inputs.shape[0]
        total_item_ct += inputs.shape[0]

    total_test_loss = 0
    total_test_acc = 0
    test_item_ct = 0
    # testing loop
    with torch.no_grad():
        for inputs, label in test_loader:
            inputs, label = inputs.to(device), label.to(device)
            
            pred = res18(inputs)
            loss = loss_fn(pred,label)
            accuracy = compute_accuracy(pred.cpu().detach().numpy(),label.cpu().detach().numpy())

            total_test_loss += loss.item()
            total_test_acc += accuracy * inputs.shape[0]
            test_item_ct += inputs.shape[0]
    
    scheduler.step()
    
    print(f'Epoch: {epoch}')
    print(f'Train Loss: {total_loss} | Train Accuracy: {total_acc/total_item_ct}')
    print(f'Test Loss: {total_test_loss} | Test Accuracy: {total_test_acc/test_item_ct}')

    if (epoch+1) % 10 == 0:
        print('Saving model...')
        torch.save(res18.state_dict(), f'saved_models/ResNet18-CIFAR10-Epoch-{epoch+1}.pth')

Epoch: 0
Train Loss: 598.971800327301 | Train Accuracy: 0.43682
Test Loss: 68.71418333053589 | Test Accuracy: 0.5038
Epoch: 1
Train Loss: 528.1632211208344 | Train Accuracy: 0.51028
Test Loss: 62.25023603439331 | Test Accuracy: 0.5414
Epoch: 2
Train Loss: 463.2668458223343 | Train Accuracy: 0.57562
Test Loss: 52.543795466423035 | Test Accuracy: 0.6299
Epoch: 3
Train Loss: 429.01760429143906 | Train Accuracy: 0.61196
Test Loss: 49.915645480155945 | Test Accuracy: 0.6499
Epoch: 4
Train Loss: 397.028099834919 | Train Accuracy: 0.64304
Test Loss: 48.7531573176384 | Test Accuracy: 0.6617
Epoch: 5
Train Loss: 376.5550960302353 | Train Accuracy: 0.6655
Test Loss: 45.01992005109787 | Test Accuracy: 0.6896
Epoch: 6
Train Loss: 365.116415143013 | Train Accuracy: 0.67546
Test Loss: 42.47970521450043 | Test Accuracy: 0.7112
Epoch: 7
Train Loss: 346.35762202739716 | Train Accuracy: 0.69282
Test Loss: 44.59938019514084 | Test Accuracy: 0.6918
Epoch: 8
Train Loss: 335.948222219944 | Train Accuracy: 0